In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Using Plotly and Geopandas to make intresting plots 

## Introduction
**In this jupyter notebook I will plot a Geo scatter map, Histogram, Box plot and Bar plot depicting Air Quality level all over Europe**

I have used the in-built functions of geopandas and plotly to do so. 

I used European air quality level data that was available in geojson format and I also used a geojson file containing map of all countries of Europe

The Description can be found [here](http://opendata.esri.es/datasets/datos-de-calidad-del-aire?geometry=146.718%2C88.176%2C11.718%2C90.000)

## Index
* Getting data 
* Importing the necessary libraries to undertake this project
* Calculating Basic statistics by using built in methods GeoPandas Dataframe
* Generating Histogram and understanding distribution of PM 2,5
* Creating a box plot to detect outliers
* Examining relationship between quality of air and altitude by means of correlation and scatter plot
* Further examination of the data
* Plotting a Geo-Scatter map
* Merging Countries dataframe with airquality dataframe
* Again Plotting a Geo-Scatter map, but this time focusing on countries
* Plotting countries with high level of pollution
* Bar plot of all countries with aggregated average of PM 2,5 levels
* Bar plot of all countries with max value registered of PM 2,5 levels

## Getting data

First I am going to get data using Curl command and save it in a file called data.geojson

The link to download the data is [here](https://opendata.arcgis.com/datasets/784fc60a00fa41cb9babb52023cb2db3_0.geojson)

In [2]:
!curl -o data.geojson https://opendata.arcgis.com/datasets/784fc60a00fa41cb9babb52023cb2db3_0.geojson

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  236k    0  236k    0     0      0      0 --:--:--  0:00:01 --:--:--     0  0   147k      0 --:--:--  0:00:01 --:--:--  147k


Also I used the geojson map data of all european countries found [here](https://geojson-maps.ash.ms/) 

## Importing the necessary libraries to undertake this project

We will use the new Plotly API called Plotly express

In [3]:
import geopandas as gpd

In [4]:
import plotly.express as px

In [5]:
import pandas as pd

## Calculating Basic statistics by using built in methods GeoPandas Dataframe

**Let us the explore the air quality data set**

Geopandas enables us to read any geojson file with its **read_file** command

In [6]:
df_air_quality = gpd.read_file("data.geojson")

The good thing about GeoPandas Dataframe is that it enables us to use various Pandas functions

In [7]:
df_air_quality.head()

,OBJECTID,EoICode,StationNam,Longitude,Latitude,Altitude,StationTyp,StationAre,avg15,geometry
0,1,AT0ENK1,Enzenkirchen im Sauwald,13.671140,48.391720,525.0,background,rural-regional,13.353289,POINT (13.67114 48.39172)
1,2,AT0ZOE2,Zöbelboden im Reichraminger Hintergebirge - Wi...,14.441389,47.838611,899.0,background,rural-remote,7.257276,POINT (14.44139 47.83861)
2,3,AT30407,Glinzendorf im Marchfeld,16.636944,48.236667,150.0,background,rural-near_city,14.789538,POINT (16.63694 48.23667)
3,4,AT31902,Zwentendorf im Tullnerfeld,15.903611,48.331111,200.0,background,rural,14.524676,POINT (15.90361 48.33111)
4,5,AT4S108,Grünbach bei Freistadt,14.574722,48.531111,918.0,background,rural-regional,9.099422,POINT (14.57472 48.53111)


Number of Unique value in each column

In [8]:
df_air_quality.nunique()

OBJECTID      710
EoICode       710
StationNam    710
Longitude     707
Latitude      707
Altitude      343
StationTyp      1
StationAre      7
avg15         710
geometry      707
dtype: int64

Checking the CRS of the dataframe

In [9]:
df_air_quality.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

**Before we make the geoscatter plot, let us explore the other plotly's features to understand the distribution of PM 2,5**

setting pandas backend to plotly from matplotlib

In [10]:
pd.options.plotting.backend = "plotly"

## Generating Histogram and understanding distribution of PM 2,5

Here I used the **histogram** function of Plotly express

In [12]:
fig = px.histogram(df_air_quality, x="avg15")

fig.update_layout(
    title={
        'text': "Distribution of PM 2,5 registered over various European stations",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(showgrid=False)  # turning off the grid
fig.show()

![title](histogram.png)

**It is a bit skewed to the right**

## Creating a box plot to detect outliers

Here I used the **box** function of Plotly express

In [13]:
fig = px.box(data_frame = df_air_quality, y = "avg15")
fig.update_layout(
    title={
        'text': "Detecting Outliers of PM 2,5 registered over various European stations",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(showgrid=False)
fig.show()

![title](boxplot.png)

**Values above 27 is considered an outlier**

## Examining relationship between quality of air and altitude by means of correlation and scatter plot


**Let's see if there is a strong correlation between altitude and PM 2,5 levels**

In [15]:
df_air_quality[["avg15","Altitude"]].corr()

,avg15,Altitude
avg15,1.000000,-0.132069
Altitude,-0.132069,1.000000


**There is a week negative correlation**

**The scatter plot is going to confirm the same**

Here is I used the **scatter** function of Plotly express

In [16]:
fig = px.scatter(data_frame = df_air_quality, y="avg15", x="Altitude")
fig.update_layout(
    title={
        'text': "Distribution of PM 2,5 registered over various Altitudes",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_traces(marker=dict(color="crimson"))
fig.update_traces(marker={"opacity": 0.5})  # Thus we see which combination of PM 2,5 and Altitude values exists the most
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

![title](scatterplot.png)

#### For higher altitude, air quality is a lot better than that found for lower altitude regions and it is due to the fact that normally lower altitude regions have higher population

## Further examination of the data

**Where are most of the stations?**

In [18]:
df_air_quality["StationAre"].value_counts()

urban              406
suburban           137
rural              112
rural-regional      36
rural-near_city      8
rural-remote         7
rural-nearcity       4
Name: StationAre, dtype: int64

#### Most of the Stations are located in urban regions but do they have poor air quality?

In [19]:
df_air_quality.groupby("StationAre")["avg15"].mean()

StationAre
rural              11.455986
rural-near_city    16.169133
rural-nearcity     18.019500
rural-regional      9.318929
rural-remote        8.609008
suburban           14.612287
urban              15.620849
Name: avg15, dtype: float64

#### On an average urban areas have a better air quality than rural areas that near the city

In [20]:
df_air_quality[df_air_quality["StationAre"] == "urban"][["avg15"]].describe()

,avg15
count,406.000000
mean,15.620848
std,6.146706
min,3.677026
25%,11.674706
50%,13.853458
75%,19.242335
max,36.010959


**But in Urban areas the PM 2,5 level can go as high as 36!**

## Plotting a Geo-Scatter map

**Let's plot the geoscatter map**

I used **scatter_mapbox** function of plotly express to plot the location of all stations present all over Europe, size of bubble indicates the level of PM 2,5. I set the opacity to 0.4.

In [25]:
fig = px.scatter_mapbox(df_air_quality, lat="Latitude", lon="Longitude", hover_name="StationNam", hover_data=["Altitude"],
                        color="StationAre", zoom=4, height=900,size="avg15",size_max=12, opacity = 0.4,width=1400)
fig.update_layout(mapbox_style='carto-positron')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(title_text="Air quality level in Europe 2018")
fig.show()

![title](geoscatter2.png)

Focusing on Terrain by selecting mapbox_style to ****

In [27]:
fig = px.scatter_mapbox(df_air_quality, lat="Latitude", lon="Longitude", hover_name="StationNam", hover_data=["Altitude"],
                        color="StationAre", zoom=4, height=900,size="avg15",size_max=12, opacity =0.4,width=1400)
fig.update_layout(mapbox_style='stamen-terrain')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(title_text="Air quality level in Europe 2018")
fig.show()

![title](terrain.png)

**Larger Bubbles can be found in regions having lower altitude**

## Merging Countries dataframe with airquality dataframe
Now I am going to read the europe geojson file

In [28]:
df_europe = gpd.read_file("europe.geo.json")

In [29]:
df_europe.head()

,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,adm0_a3,...,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,filename,geometry
0,1,Admin-0 country,6,Albania,ALB,0,2,Sovereign country,Albania,ALB,...,Europe,Southern Europe,Europe & Central Asia,7,7,4,-99,1,ALB.geojson,"POLYGON ((20.59025 41.85540, 20.46318 41.51509..."
1,1,Admin-0 country,4,Austria,AUT,0,2,Sovereign country,Austria,AUT,...,Europe,Western Europe,Europe & Central Asia,7,7,5,-99,1,AUT.geojson,"POLYGON ((16.97967 48.12350, 16.90375 47.71487..."
2,1,Admin-0 country,2,Belgium,BEL,0,2,Sovereign country,Belgium,BEL,...,Europe,Western Europe,Europe & Central Asia,7,7,5,-99,1,BEL.geojson,"POLYGON ((3.31497 51.34578, 4.04707 51.26726, ..."
3,1,Admin-0 country,4,Bulgaria,BGR,0,2,Sovereign country,Bulgaria,BGR,...,Europe,Eastern Europe,Europe & Central Asia,8,8,5,-99,1,BGR.geojson,"POLYGON ((22.65715 44.23492, 22.94483 43.82379..."
4,1,Admin-0 country,4,Belarus,BLR,0,2,Sovereign country,Belarus,BLR,...,Europe,Eastern Europe,Europe & Central Asia,7,7,5,-99,1,BLR.geojson,"POLYGON ((23.48413 53.91250, 24.45068 53.90570..."


There is no need to keep all the columns, I am only going to keep admin(country) and geometry

In [30]:
df_europe = df_europe[["admin","geometry"]]

Now I am going to merge the geopandas dataframe containing coordinates of all european countries and the geopandas dataframe that has information of air quality

In [31]:
df_merged = gpd.sjoin(df_air_quality, df_europe, how="inner", op='intersects')

In [32]:
df_merged.head() 

,OBJECTID,EoICode,StationNam,Longitude,Latitude,Altitude,StationTyp,StationAre,avg15,geometry,index_right,admin
0,1,AT0ENK1,Enzenkirchen im Sauwald,13.671140,48.391720,525.0,background,rural-regional,13.353289,POINT (13.67114 48.39172),1,Austria
1,2,AT0ZOE2,Zöbelboden im Reichraminger Hintergebirge - Wi...,14.441389,47.838611,899.0,background,rural-remote,7.257276,POINT (14.44139 47.83861),1,Austria
2,3,AT30407,Glinzendorf im Marchfeld,16.636944,48.236667,150.0,background,rural-near_city,14.789538,POINT (16.63694 48.23667),1,Austria
3,4,AT31902,Zwentendorf im Tullnerfeld,15.903611,48.331111,200.0,background,rural,14.524676,POINT (15.90361 48.33111),1,Austria
4,5,AT4S108,Grünbach bei Freistadt,14.574722,48.531111,918.0,background,rural-regional,9.099422,POINT (14.57472 48.53111),1,Austria


## Bar plot of all countries with aggregated average of PM 2,5 levels

But I will only plot for those countries who at least have 5 stations

In [33]:
# making a list of countries having more than 5 stations
countries_over_5 = df_merged.groupby("admin").size().apply(lambda g: g>=5)


countries_over_5_df = df_merged[df_merged['admin'].isin(countries_over_5[countries_over_5].index)]

In [37]:
fig1 = px.bar(countries_over_5_df.groupby("admin")["avg15"].mean(),  text=countries_over_5_df.groupby("admin").size(),width=1300)
fig1.update_layout(
    title={
        'text': "PM 2,5 levels registered on average by countries",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig1.show()


![title](barplot_mean.png)

**On an average the worst air quality is found in Poland, Italy and Slovakia**

## Bar plot of all countries with max value registered of PM 2,5 levels

Here I have included all countries

In [38]:
fig2 = px.bar(data_frame = df_merged.groupby("admin")["avg15"].max().sort_values(),width = 1300)
fig2.update_layout(
    title={
        'text': "Maximum PM 2,5 levels registered by countries",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig2.show()

![title](barplot_max.png)

**Some of the stations that have the worse air quality are in Poland, Czech Republic and Italy**

## Plotting countries with high level of pollution

I am going to first plot a map of Poland and I am going to Increase the **"zoom" argument to 6**, so that we only see Poland when the map is generated, I also increased the **size_max** argument to get a better contrast

In [40]:
fig = px.scatter_mapbox(df_merged[df_merged["admin"]=="Poland"], lat="Latitude", lon="Longitude", 
                        hover_name="StationNam",hover_data=["Altitude","admin"],
                        color="StationAre", zoom=6, height=900,size="avg15",size_max=20, title ="Air quality level in Poland 2018" ,width=1300)
fig.update_layout(mapbox_style='carto-positron')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

![title](poland.png)

**Southern part of Poland appears to have a poor quality of Air, as it has more urban centers**

I am going to plot Italy, I am going to decrease the value of **zoom to 5** just so that Italy is in focus when the map is generated, I again made **size_max = 15** 

In [41]:
fig = px.scatter_mapbox(df_merged[df_merged["admin"]=="Italy"], lat="Latitude", lon="Longitude", hover_name="StationNam", hover_data=["Altitude"],
                        color="StationAre", zoom=5, height=900,size="avg15",size_max=15,title  ="Air quality level in Italy 2018",width=1400)
fig.update_layout(mapbox_style='carto-positron')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

![title](italy.png)

#### In case of Italy, the northern region has higher level of pollution

## <span style="color:green">*I hope this Notebook helps you in undertaking your own assignment, and I will be writing more posts regarding this topic :)*
</span>